In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds

In [2]:
(ds_train, ds_test), ds_info = tfds.load(
    'mnist_corrupted',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/60000 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/mnist_corrupted/identity/1.0.0.incompleteTOCKL3/mnist_corrupted-train.tfrecord…

Generating test examples...:   0%|          | 0/10000 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/mnist_corrupted/identity/1.0.0.incompleteTOCKL3/mnist_corrupted-test.tfrecord*…

Dataset mnist_corrupted downloaded and prepared to ~/tensorflow_datasets/mnist_corrupted/identity/1.0.0. Subsequent calls will reuse this data.


In [3]:
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

In [4]:
ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)

In [6]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(512, kernel_initializer='he_normal', activation='relu'),
  tf.keras.layers.Dropout(0.2), 
  tf.keras.layers.Dense(512, kernel_initializer='glorot_uniform', activation='elu'), 
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(256,kernel_initializer='glorot_normal', activation='relu'), 
  tf.keras.layers.Dropout(0.2), 
  tf.keras.layers.BatchNormalization() 
     ])
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

model.fit(
    ds_train,
    epochs=7,
    validation_data=ds_test,
)

Epoch 1/7
469/469 [==============================] - 13s 25ms/step - loss: 2.7125 - sparse_categorical_accuracy: 0.7033 - val_loss: 1.3895 - val_sparse_categorical_accuracy: 0.9663
Epoch 2/7
469/469 [==============================] - 10s 22ms/step - loss: 1.5597 - sparse_categorical_accuracy: 0.7849 - val_loss: 0.6508 - val_sparse_categorical_accuracy: 0.9745
Epoch 3/7
469/469 [==============================] - 10s 22ms/step - loss: 1.1652 - sparse_categorical_accuracy: 0.7917 - val_loss: 0.3139 - val_sparse_categorical_accuracy: 0.9766
Epoch 4/7
469/469 [==============================] - 10s 22ms/step - loss: 0.9814 - sparse_categorical_accuracy: 0.7957 - val_loss: 0.2182 - val_sparse_categorical_accuracy: 0.9799
Epoch 5/7
469/469 [==============================] - 10s 20ms/step - loss: 0.8516 - sparse_categorical_accuracy: 0.8010 - val_loss: 0.1822 - val_sparse_categorical_accuracy: 0.9806
Epoch 6/7
469/469 [==============================] - 10s 20ms/step - loss: 0.7600 - sparse_cate

In [7]:
 model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 784)               0         
                                                                 
 dense_3 (Dense)             (None, 512)               401920    
                                                                 
 dropout_3 (Dropout)         (None, 512)               0         
                                                                 
 dense_4 (Dense)             (None, 512)               262656    
                                                                 
 dropout_4 (Dropout)         (None, 512)               0         
                                                                 
 dense_5 (Dense)             (None, 256)               131328    
                                                                 
 dropout_5 (Dropout)         (None, 256)              

## ** Based on the DL course, HIT, Computer Science. **